# River masks Semi-authomatic Multi-Temporal extraction from Landsat SR images usign Google Earth Engine
### Variant of the original code that uses a service account

### Niccolò Ragno, Riccardo Bonanomi, Marta Crivellaro, Alfonso Vitti, Guido Zolezzi and Marco Tubino
* Publication corresponding Author: Niccolò Ragno,  niccolo.ragno@unitn.it
* GEE code corresponding Author: Marta Crivellaro,  marta.crivellaro@unitn.it

GEE Python installation: https://developers.google.com/earth-engine/guides/python_install

In [ ]:
#libraries import
import os, sys, glob, math,subprocess,tarfile,shutil
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import linalg as la
from functions import ndvi,mndwi,addindex,areaImg,maxValue,ndviMap,mndwiMap
import ee
import geemap
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials
import time

In [ ]:
# Google Earth Engine authentication - see readme for details
from credentials_script import get_credentials
service_account, credentials_folder, credentials_file = get_credentials()
credentials = ee.ServiceAccountCredentials(service_account, credentials_folder + credentials_file)
ee.Initialize(credentials)

# Google drive autentiucation - see readme for details
gauth = GoogleAuth()
scopes = ['https://www.googleapis.com/auth/drive']
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_folder+credentials_file, scopes=scopes)
drive = GoogleDrive(gauth)

In [ ]:
# specify the input data
river_name = 'Kuk'

#1 - Define region(S) of interest (roi) as rectangular extents
#EPSG: 32604 Kuk Alaska
river_geom = ee.Geometry.Polygon([
    [[-159.2000, 70.0365],
     [-159.0765, 70.2179],
     [-159.6402, 70.1556],
     [-159.7777, 70.0919],
     [-159.2000, 70.0365]]])
EPSG_code = 'EPSG:32604'

In [ ]:
# output data folder
# local
output_folder = 'D:/Documents/PhD/Codici/PyRIS/GEE-masks/output/' + river_name + '/'
os.makedirs(output_folder, exist_ok=True)

# Google Drive
PM_drive_folder = river_name + '-' + 'Permafrost_rivermask_PM'
regular_drive_folder = river_name + '-' + 'Permafrost_rivermask'

### Create an interactive map
The default basemap is _Google Maps_. Additional basemaps can be added using the ``Map.add_basemap()`` function.

In [ ]:
Map = geemap.Map(center=[19.76,40.41], zoom=22)
print('Done!')

In [ ]:
fcGeom = river_geom
roi = fcGeom 
Map.centerObject(roi)
Map

Standardise band names, merge Landsat data:

In [ ]:
#Standardise band names, merge Landsat data:
bn8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B6', 'SR_QA_AEROSOL', 'SR_B5', 'SR_B7', 'QA_PIXEL']
bn7 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B5', 'SR_CLOUD_QA', 'SR_B4', 'SR_B7','QA_PIXEL']
bn5 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B5', 'SR_CLOUD_QA', 'SR_B4', 'SR_B7','QA_PIXEL']
#standard bans:
bnL = ['uBlue', 'Blue', 'Green', 'Red', 'Swir1', 'BQA', 'Nir', 'Swir2','qa_pixel']

## defining cloudmask function for landsat 7 and 8 only
# This function masks the input with a threshold on the simple cloud score.
# Observe that the input to simpleCloudScore() is a single Landsat TOA scene. 
# Also note that simpleCloudScore() adds a band called ‘cloud’ to the input image. 
# The cloud band contains the cloud score from 0 (not cloudy) to 100 (most cloudy).
def cloudMask(img):
    cloudscore = ee.Algorithms.Landsat.simpleCloudScore(img).select('cloud')
    return img.updateMask(cloudscore.lt(10))

def maskClouds(image):
    
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    
    qa = image.select('qa_pixel')
    mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0)))
    
    return image.updateMask(mask)

#calling LS Surface Reflectance image collections 
ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filter(ee.Filter.lt('CLOUD_COVER',15)).select(bn5, bnL)
ls7 = (ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")  \
  .map(cloudMask)  \
  .filterDate('1999-04-15', '2003-05-30')  \
  .select(bn7, bnL))
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(ee.Filter.lt('CLOUD_COVER', 15)).select(bn8, bnL)
#merging LS 5 and 8 dataset
ls = ls5.merge(ls8).sort('system:start', True).filter(ee.Filter.calendarRange(5,9,'month'))

In [ ]:
#functions definition
def ErosDil(image):
    kernel = ee.Kernel.square(2)
    opened = image.focalMin(kernel=kernel, iterations=2).focalMax(kernel=kernel, iterations=2)
    return opened

def ClassifyWater(imgIn, method = 'Jones2019'):
    if method == 'Jones2019':
        from functions_waterClassification_Jones2019median import ClassifyWaterJones2019
        return(ClassifyWaterJones2019(imgIn))
    elif method == 'Zou2018':
        from functions_waterClassification_Zou2018median import ClassifyWaterZou2018
        return(ClassifyWaterZou2018(imgIn))
    
def RGBtoHSV (Image):
    sat = Image.select(['Red','Green','Blue']).divide(65455).rgbToHsv().select(['saturation'])
    return Image.addBands(sat)

def histogram(image):
    # Compute the histogram of the NIR band.  The mean and variance are only FYI.
    polygon = ee.Geometry(image.geometry())
    histogram = image.reduceRegion(
        **{
            'reducer': ee.Reducer.histogram(255, 2),
            'geometry': polygon,
            'scale': 15,
            'bestEffort': True,
        }
    )
    return histogram

# Return the DN that maximizes interclass variance in B5 (in the region).
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)

    indices = ee.List.sequence(1, size)

    # Compute between sum of squares, where each mean partitions the data.

    def func_xxx(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = (
            aMeans.multiply(aCounts)
            .reduce(ee.Reducer.sum(), [0])
            .get([0])
            .divide(aCount)
        )
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
            bCount.multiply(bMean.subtract(mean).pow(2))
        )

    bss = indices.map(func_xxx)

    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

def extract_ac(imagesat,imagemndwi, t_sat, t_mndwi):
    ac = imagesat.select('saturation_median').gte(t_sat).Or(imagemndwi.select('mndwi_median').gte(t_mndwi)).selfMask()
    return ee.Image(ac)

def stdLocal (image, roi): 
    geom = roi.geometry()
    std_value = image.clip(geom).reduceRegion(**{
        'reducer': ee.Reducer.stdDev(),
        'geometry': geom,
        'scale': 30,
        'maxPixels': 1e12,
        'tileScale': 16
    })
    return std_value

In [ ]:
#PERONA MALIK FILTER
# Perona malik filter
# I(n+1, i, j) = I(n, i, j) + Lambda * (cN * dN(I) + cS * dS(I) + cE * dE(I), cW * dW(I))
#**
#Perona-Malik (anisotropic diffusion) convolution
#by Gennadii Donchyts see https://groups.google.com/forum/#!topic/google-earth-engine-developers/a9W0Nlrhoq0
#I(n+1, i, j) = I(n, i, j) + lambda * (cN * dN(I) + cS * dS(I) + cE * dE(I), cW * dW(I))
#iter: Number of interations to apply filter
#K: kernal size
#method: choose method 1 (default) or 2
# Returns: image 
#
def peronaMalikFilter(I, iter, K, method, l):
    dxW = ee.Kernel.fixed(3, 3, [[ 0,  0,  0], [ 1, -1,  0], [ 0,  0,  0]])
    dxE = ee.Kernel.fixed(3, 3, [[ 0,  0,  0], [ 0, -1,  1], [ 0,  0,  0]])
    dyN = ee.Kernel.fixed(3, 3, [[ 0,  1,  0], [ 0, -1,  0], [ 0,  0,  0]])
    dyS = ee.Kernel.fixed(3, 3, [[ 0,  0,  0], [ 0, -1,  0], [ 0,  1,  0]])
    
    Lambda = l 
    
    k1 = ee.Image(-1.0/K)
    k2 = ee.Image(K).multiply(ee.Image(K))
    
    for i in range(0, iter):
        dI_W = I.convolve(dxW)
        dI_E = I.convolve(dxE)
        dI_N = I.convolve(dyN)
        dI_S = I.convolve(dyS)
        
        if method == 1:
            cW = dI_W.multiply(dI_W).multiply(k1).exp()
            cE = dI_E.multiply(dI_E).multiply(k1).exp()
            cN = dI_N.multiply(dI_N).multiply(k1).exp()
            cS = dI_S.multiply(dI_S).multiply(k1).exp()
        elif method == 2:
            cW = ee.Image(1.0).divide(ee.Image(1.0).add(dI_W.multiply(dI_W).divide(k2)))
            cE = ee.Image(1.0).divide(ee.Image(1.0).add(dI_E.multiply(dI_E).divide(k2)))
            cN = ee.Image(1.0).divide(ee.Image(1.0).add(dI_N.multiply(dI_N).divide(k2)))
            cS = ee.Image(1.0).divide(ee.Image(1.0).add(dI_S.multiply(dI_S).divide(k2)))
        I = I.add(ee.Image(Lambda).multiply(cN.multiply(dI_N).add(cS.multiply(dI_S)).add(cE.multiply(dI_E)).add(cW.multiply(dI_W))))
    
    return I

### Cycle on years to export annual domain mask

In [ ]:
dates = [1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
         1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
         2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
         2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
dates_out = dates[:]
ACy = {}
data= []

roi = fcGeom
for year in dates:
    sDate_T1 = str(year)+"-05-01"; 
    eDate_T1 = str(year)+"-09-30";
    #Sort by:  roi, date:
    collection= ls \
        .filterBounds(roi) \
        .sort('system:start', True) \
        .filterDate(sDate_T1,eDate_T1)

    # Create a list of image objects.
    imageList = collection.toList(100);
    Collection = collection.map(ndviMap).map(mndwiMap).map(RGBtoHSV)
    median = Collection.reduce(ee.Reducer.median())#.reproject(ee.Projection(EPSG_code)) #qui è bene cambiare con la proiezione WGS 84 UTM LOCALE
    maxi = Collection.reduce(ee.Reducer.percentile([90]))#.reproject(ee.Projection(EPSG_code))
    mini = Collection.reduce(ee.Reducer.min())#.reproject(ee.Projection(EPSG_code))

    ndvimap_median=median.select('ndvi_median').clip(roi)
    ndvimap_90p=maxi.select('ndvi_p90').clip(roi)
    mndwimap_median=median.select('mndwi_median').clip(roi)
    
    satmap_90p=maxi.select('saturation_p90').clip(roi)
   
    ndwimap_90p=maxi.select('mndwi_p90').clip(roi)


    pm_mndwi_0_3 = peronaMalikFilter(mndwimap_median, 5, 2, 1, 0.3)
  
    otsu_mndwi = otsu(histogram(mndwimap_median).get('mndwi_median'))
    otsu_sat = otsu(histogram(satmap_90p).get('saturation_p90'))
    otsu_mndwiPM = otsu(histogram(pm_mndwi_0_3).get('mndwi_median'))
    
    veg1 = ndvimap_90p.select('ndvi_p90').gte(0.15)
    water3 =  satmap_90p.gt(otsu_sat).Or(ee.Image(mndwimap_median.select('mndwi_median')
                                                  .gte(otsu_mndwi)).And(veg1.lt(1)))

    waterPM3 = satmap_90p.gt(otsu_sat).Or(pm_mndwi_0_3.select('mndwi_median')
                                          .gte(otsu_mndwiPM)).And(veg1.lt(1))

    area_raw_N = areaImg(water3.remap(ee.List([0]),ee.List([1])))
    area_pm_N = areaImg(waterPM3.remap(ee.List([0]),ee.List([1])))
    
    try:
        # extract the value as a number
        area_raw_number = area_raw_N.getNumber('remapped').getInfo()
        area_pm_number = area_pm_N.getNumber('remapped').getInfo()
        #dataframe with extracted areas and thresholds info creation and compiling
        data.append(dict(zip(('year','area_raw_number','area_pm_number','%d','t_nmndwi','t_PMmndwi'),
                            (str(year),area_raw_number,area_pm_number,(100*((area_raw_number-area_pm_number)/area_raw_number)),otsu_mndwi.getInfo(),otsu_mndwiPM.getInfo(),))))
        
        #Export the images, specifying scale and region.
        task = ee.batch.Export.image.toDrive(**{
                'image': waterPM3.clip(fcGeom),
                'description': river_name + str(year),
                'folder': PM_drive_folder,
                'scale': 30,
                'crs': EPSG_code,
                'region': fcGeom

            })
        task.start()

        task = ee.batch.Export.image.toDrive(**{
                'image': water3.clip(fcGeom),
                'description': river_name + str(year),
                'folder': regular_drive_folder,
                'scale': 30,
                'crs': EPSG_code,
                'region': fcGeom

            })
        task.start()
                    
        print(str(year) + ' Done!')
    except:
        print(str(year) + ' Error!')
        dates_out.remove(year)

print('The year where there are data are:')
print(dates_out)

In [ ]:
# saving the dataframe as csv file
df = pd.DataFrame(data)
df.to_csv(output_folder + river_name + '_stats.csv')
df

In [ ]:
# check if exporting task are yet to start or are still running
export_task  = [dict for dict in ee.data.listOperations() if dict['metadata']['type'] == 'EXPORT_IMAGE']
running_task = [dict for dict in export_task if dict['metadata']['state'] == 'RUNNING']
ready_task   = [dict for dict in export_task if dict['metadata']['state'] == 'READY']

while not (not running_task and not ready_task):
    print('There are still %d running tasks and %d task to start. Cheching again in a minute.', (len(running_task), len(ready_task)))    
    time.sleep(60)

# download the files from Google Drive and delete them afterwards
# list all folders in the root Drive folder
folder_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

if not folder_list:
    print('No folders found in the root folder')
else:
    print('Downloading files from Google Drive')

    # iterate over all folders
    for folder in folder_list:
        if folder['title'] == regular_drive_folder:
            saving_folder = output_folder + 'rivermask/'
        elif folder['title'] == PM_drive_folder:
            saving_folder = output_folder + 'rivermask_PM/'
        else:
            print('Deleting folder: %s, id: %s' % (folder['title'], folder['id']))
            folder.Delete()
            continue
        
        os.makedirs(saving_folder, exist_ok=True)
        print('\ntitle: %s, id: %s' % (folder['title'], folder['id']))

        # list and download all files in the folder
        file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % folder['id']}).GetList()

        for file in file_list:
            filename = file['title']
            print('title: %s, id: %s' % (file['title'], file['id']))
            
            # download file into working directory (in this case a tiff-file)
            file.GetContentFile(saving_folder + filename, mimetype="image/tiff")

            # delete file afterwards to keep the Drive empty
            file.Delete()
            
        folder.Delete()